<a href="https://colab.research.google.com/github/andrearakel/Mastersverkefni/blob/main/MyResNetFeatureExtractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from collections import namedtuple
import csv
import sys
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import os
import itertools
# import the necessary packages
from collections import namedtuple
import numpy as np
import cv2
from PIL import Image, ImageOps
# importing XML parsing library for parsing the data
import xml.etree.ElementTree as ET
from pathlib import Path
import pandas as pd
from keras_preprocessing.image import ImageDataGenerator
from collections import Counter
import seaborn as sns
from keras import backend as K
from keras.callbacks import Callback
from keras.layers import Dense
from keras.models import Model
from sklearn.metrics import(f1_score, precision_score, recall_score)
from sklearn.metrics import multilabel_confusion_matrix

import numpy as np
from sklearn.linear_model import LogisticRegression
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
from tqdm import tqdm

import torch

import torchvision.transforms as transforms
import cv2
from sklearn.model_selection import train_test_split

# import the necessary packages
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from imutils import paths
import numpy as np
import pickle
import random
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
annotations_dir = Path('/content/drive/MyDrive/Data/B1/labels/').expanduser()
images_dir = Path('/content/drive/MyDrive/Data/Small data/PNG myndir/').expanduser()

In [ ]:
%cd '/content/drive/MyDrive/Data/Small data/'

IMG_DIR = 'PNG myndir/'

/content/drive/MyDrive/Data/Small data


In [ ]:
import torch

BATCH_SIZE = 4 # increase / decrease according to GPU memeory
RESIZE_TO = 1024 # resize the image for training and transforms
NUM_EPOCHS = 100 # number of epochs to train for
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# training images and XML files directory
TRAIN_DIR = IMG_DIR
VAL_DIR = IMG_DIR

In [ ]:
#Get labels
def build_labels(path_to_xml_annotation, unique_labels, annotation_file):
  root = ET.XML(path_to_xml_annotation)
  labels = set() if unique_labels else []
  labels_add = labels.add if unique_labels else labels.append

  for i, child in enumerate(root):
        if child.tag == 'object':
            for subchild in child:
                if subchild.tag == 'name':
                    labels_add(subchild.text)
      

  return annotation_file.stem[:-2], list(labels)


def get_labels(annotations_dir, unique_labels=True):
    for annotation_file in annotations_dir.iterdir():
        with open(annotation_file) as f:
            yield build_labels(f.read(), unique_labels, annotation_file)

In [ ]:
img_metadata = pd.DataFrame(get_labels(annotations_dir), columns=['filename', 'labels'])
print(f'Found {len(img_metadata)} images')
img_metadata.sample(5)

Found 270 images


,filename,labels
24,29_2021-10-04_12.14.36.000_out_,[worm]
208,215_2021-10-04_14.08.45.000_out_,[worm]
227,234_2021-10-04_14.16.57.000_out_,[worm]
8,18_I0018_out_,[worm]
213,220_2021-10-04_14.11.11.000_out_,[worm]


In [ ]:
train_set,test_set = train_test_split(img_metadata,test_size=0.2, random_state=42)

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, df):
        self.images = df["filename"].tolist()
        self.caption = df["labels"].tolist()

    def __len__(self):
        return len(self.caption)

    def __getitem__(self, idx):
  
        image = load_img((str(images_dir) + '/' + self.images[idx] + '07.png'), target_size=(224,224))
        image = img_to_array(image)
        image = np.expand_dims(image, axis=0)
        image = np.array(image, dtype='float64')
        image = preprocess_input(image)
        
        caption = str(self.caption[idx])
        if caption == "['worm']":
         caption = 1
        else:
         caption = 0

        return image,caption
    

In [ ]:
train_dataset_B1 = ImageDataset(train_set)
test_dataset_B1 = ImageDataset(test_set)

In [ ]:
train_dataset_B2 = ImageDataset(train_set)
test_dataset_B2 = ImageDataset(test_set)

In [ ]:
train_dataset_B3 = ImageDataset(train_set)
test_dataset_B3 = ImageDataset(test_set)

In [ ]:
train_dataset_B4 = ImageDataset(train_set)
test_dataset_B4 = ImageDataset(test_set)

In [ ]:
train_dataset_B5 = ImageDataset(train_set)
test_dataset_B5 = ImageDataset(test_set)

In [ ]:
train_dataset_B6 = ImageDataset(train_set)
test_dataset_B6 = ImageDataset(test_set)

In [ ]:
train_dataset_B7 = ImageDataset(train_set)
test_dataset_B7 = ImageDataset(test_set)

In [ ]:
model = ResNet50(weights="imagenet", include_top=False)


94781440/94765736 [==============================] - 2s 0us/step


In [ ]:
def collate_fn(batch):
    """
    To handle the data loading as different images may have different number 
    of objects and to handle varying size tensors as well.
    """
    return tuple(zip(*batch))

In [ ]:
def get_features(dataset):
  all_features = []
  all_labels = []

  with torch.no_grad():
    for images, labels in tqdm(DataLoader(dataset, collate_fn=collate_fn)):
      features = model.predict(images)
      features = features.flatten()
      features = np.array(features, dtype="float")
      all_features.append(features)
      all_labels.append(labels)

  return all_features, all_labels


In [ ]:
train_features_B1, train_labels_B1 = get_features(train_dataset_B1)
test_features_B1, test_labels_B1 = get_features(test_dataset_B1)

100%|██████████| 54/54 [01:22<00:00,  1.52s/it]


In [ ]:
train_features_B2, train_labels_B2 = get_features(train_dataset_B2)
test_features_B2, test_labels_B2 = get_features(test_dataset_B2)

100%|██████████| 54/54 [01:31<00:00,  1.69s/it]


In [ ]:
train_features_B3, train_labels_B3 = get_features(train_dataset_B3)
test_features_B3, test_labels_B3 = get_features(test_dataset_B3)

100%|██████████| 54/54 [01:24<00:00,  1.57s/it]


In [ ]:
train_features_B4, train_labels_B4 = get_features(train_dataset_B4)
test_features_B4, test_labels_B4 = get_features(test_dataset_B4)

In [ ]:
train_features_B5, train_labels_B5 = get_features(train_dataset_B5)
test_features_B5, test_labels_B5 = get_features(test_dataset_B5)

In [ ]:
train_features_B6, train_labels_B6 = get_features(train_dataset_B6)
test_features_B6, test_labels_B6 = get_features(test_dataset_B6)

In [ ]:
train_features_B7, train_labels_B7 = get_features(train_dataset_B7)
test_features_B7, test_labels_B7 = get_features(test_dataset_B7)

Stack feature vectors

In [ ]:
train_features = np.hstack((train_features_B1, train_features_B2, train_features_B3, train_features_B4, train_features_B5, train_features_B6, train_features_B7))
test_features = np.hstack((test_features_B1, test_features_B2, test_features_B3, test_features_B4, test_features_B5, test_features_B6, test_features_B7))

Mean of feature vectors

In [ ]:
train_features = (np.array(train_features_B1) + np.array(train_features_B2))/2
test_features = (np.array(test_features_B1) + np.array(test_features_B2))/2

Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
clf = LogisticRegressionCV(Cs = 50, random_state=0, max_iter=1000).fit(train_features, np.ravel(train_labels_B1))
pred_clf = clf.predict(test_features)
accuracy = np.mean((np.ravel(test_labels_B1) == pred_clf).astype(float)) * 100.
print(accuracy)

55.55555555555556


In [ ]:
# Model Evaluation metrics 
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
print('Accuracy Score : ' + str(accuracy_score(test_labels_B1,predictions)))
print('Precision Score : ' + str(precision_score(test_labels_B1,predictions)))
print('Recall Score : ' + str(recall_score(test_labels_B1,predictions)))
print('F1 Score : ' + str(f1_score(test_labels_B1,predictions)))

Accuracy Score : 0.42592592592592593
Precision Score : 0.4339622641509434
Recall Score : 0.9583333333333334
F1 Score : 0.5974025974025975


In [ ]:
#Logistic Regression Classifier Confusion matrix
from sklearn.metrics import confusion_matrix
print('Confusion Matrix : \n' + str(confusion_matrix(test_labels_B1,predictions)))

Confusion Matrix : 
[[15 12]
 [ 1 26]]
